# Context

Since it first began, covid-19 has taken over the world. Not only has it dramatically affected global health, but also social, economic, and cultural norms across the world. Its rippling shockwaves continue to be felt to this day. This data exploration will take a deeper look at the cases over the last few years. 

In this data exploration, I use the live WHO data at this link here:

https://covid19.who.int/data

In [2]:
import pandas as pd
import numpy as np
import requests
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers